In [ ]:
(* #use "topfind";; *)
(* #require "ppx_deriving";; *)
(* #require "ppx_jane";; *)
#require "yojson";;
#require "ppx_yojson_conv";;
(* #require "core";; *)
#require "async";;
#require "cohttp-async";;
#require "async_ssl";;
#require "ppx_let" ;;

#require "owl-top, jupyter.notebook";;
#require "jupyter-archimedes";;
#require "owl-jupyter";;
#require "owl-plplot";;

(* open Owl_jupyter;; *)
(* open Owl;; *)

(* #thread ;;
let () = Thread_safe.block_on_async_exn (fun () -> get_token "15011272962" "u123456A") *)

In [ ]:
let open Owl in
let open Owl_plplot in
let open Owl_jupyter in

let y1 = Mat.uniform 1 10 in
let y2 = Mat.uniform 10 100 in
let h = Plot.create ~m:2 ~n:1 "plot_008.png" in
Plot.subplot h 0 0;
Plot.(bar ~h ~spec:[ RGB (0,153,51); FillPattern 3 ] y1);
Plot.subplot h 1 0;
Plot.(boxplot ~h ~spec:[ RGB (0,153,51) ] y2);
Plot.output h
;;

In [ ]:
let open Owl in
let open Owl_plplot in
let open Owl_jupyter in

  let f x = Maths.sin x /. x in
  let h = Plot.create "plot_01.png" in
  Plot.set_title h "Function: f(x) = sine x / x";
  Plot.set_xlabel h "x-axis";
  Plot.set_ylabel h "y-axis";
  Plot.set_font_size h 8.;
  Plot.set_pen_size h 3.;
  Plot.plot_fun ~h f 1. 15.;
  Plot.output h;;

In [25]:
let open Owl in
let open Owl_plplot in
let open Owl_jupyter in

(* let h = Plot.create "plot_024.png" in
Plot.(plot_fun ~h ~spec:[ RGB (0,0,255); Marker "#[0x2299]"; MarkerSize 2. ] Maths.sin 0. 9.);
Plot.(plot_fun ~h ~spec:[ RGB (255,0,0); Marker "#[0x0394]"; MarkerSize 8. ] Maths.cos 0. 9.);
Plot.legend_on h [|"Sine function"; "Cosine function"|];
Plot.output h *)

let a = Maths.abs (-1.)


error: compile_error

In [21]:
let f = Owl.Maths.abs

val f : float -> float = <fun>
